In [ ]:
!pip install pytorchvideo

In [ ]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle

import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale
from torchvision.transforms._transforms_video import CenterCropVideo, NormalizeVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim



class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        print("len x" , len(self.x))
        self.y = y
        print("len y" , len(self.y))


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [ ]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60
mean = 0
std = 0



inputs =[] #x
classes = [] #y

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            
            Lambda(lambda x: x.permute(1,0,2,3)),#(frames(depth), channel, height, width) -> (channel, frames(depth), height, width)
            
            UniformTemporalSubsample(NUM_FRAMES),
            Lambda(lambda x: x.permute(1,0,2,3)),#(frames(depth), channel, height, width)
            
            Lambda(lambda x: x/255.0), 
            NormalizeVideo((mean,), (std,)),
            CenterCropVideo([720,800]),
            Lambda(lambda x: x.permute(1,0,2,3)),#(channel, frames(depth), height, width)
            
            
        ]
        
    ),
)
def get_data_info(f):
    for line in f:
        a = line.split(',')
        yield a






def load_dataloader(batch_size):
    video = EncodedVideo.from_path("../input/signdataset/p01_n000.mp4")
        

            


    with open('../input/signdataset/p01_n000.txt') as f: 
        for x in get_data_info(f):
            classes.append(int(x[0]))
            start_time = x[1]
            end_time = x[2]
            
            video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)

            
            video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))

            #print(video_data["video"].shape)
            std, mean = torch.std_mean(video_data["video"])
            #print(std, mean)
            video_data = transform( video_data)

        # Move the inputs to the desired device
            inputs.append(video_data["video"])

    signds = SignDataset(inputs, classes)
    dataloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=1)

    return dataloader
        



In [ ]:
class conv_3d(nn.Module):
    def __init__(self):
        super(conv_3d, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        

        self._features = nn.Sequential(
            self.conv1,
            self.conv2
        )


    def forward(self, x):
        out = self._features(x)
        print("conv3d", out.shape)
        out= out.reshape(out.shape[0], out.shape[1]*out.shape[2], out.shape[3], out.shape[4])
        print("reshape conv3d ",out.shape)
        return out

class conv_2d(nn.Module):
    def __init__(self):
        super(conv_2d, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(640, 640, kernel_size = 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
            
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(640, 512, kernel_size = 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size = 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
        
    
    def forward(self,x):
        out = self.conv2(self.conv1(x))
        out = self.conv3(out)
        print(out.shape)
        out = out.view(out.shape[0],-1)
        print(out.shape)
        return out


  








In [ ]:



n_classes = 60

model = nn.Sequential(OrderedDict([
    ('frontend', conv_3d()),
    ('mid', conv_2d()),
    ('fc', nn.Sequential( nn.Dropout(p=0.4), nn.Linear(135168, 1024),nn.Linear(1024,256), nn.Linear(256,60) ))
]))




# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.SGD(
  [
        {"params": model.fc.parameters(), "lr": 1e-3},
        {"params": model.mid.parameters(), "lr": 1e-5},
        {"params": model.frontend.parameters(), "lr": 1e-4},
  ],
  momentum = 0.9
)


def train(model, device, train_loader, optimizer, criterion, epoch):
    model.cuda()
    model.train()
    train_loss = 0
    correct = 0
    num_samples = 0
    
    for epoch in tqdm(range(epoch)):
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = torch.tensor(target).to(device)

    
    
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            num_samples += pred.shape[0]
            print(num_samples)
            correct += pred.eq(target.view_as(pred)).sum().item()
            print("correct till now:",correct)
        
    train_loss /= num_samples
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, num_samples,
                100. * correct / num_samples, train_loss))


dataloader = load_dataloader(batch_size=1)




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train(model,device,dataloader,optimizer,criterion,10)